    Pair RDD键值对RDD，通常用来进行聚合操作

    4.2 创建PairRDD

In [1]:
#引入并初始化pyspark
import findspark
findspark.init()

In [2]:
#python中初始化spark
from pyspark import SparkContext, SparkConf
conf = SparkConf().setMaster("local").setAppName("my app")#集群的URL，应用名
sc = SparkContext(conf = conf)

In [3]:
#创建一个名为README.MD的RDD，弹性分布式数据集
lines = sc.textFile("README.MD")

In [8]:
pairLines = lines.map(lambda x:(x.split()[0],x))
pairLines.take(1)#.collect()

[(u'Once',
  u'Once upon a time there was a baby eagle living in a nest perched on a cliff overlooking a beautiful valley with waterfalls and streams, trees and lots of little animals, scurrying1 about enjoying their lives.')]

    4.3 Pair RDD转化操作
    按功能区分：
    聚合操作
    数据分组
    连接
    数据排序

    4.3.1 聚合操作
    reduceByKey()函数，合并具有相同键的值
    combineBykey()函数，使用不同的返回类型合并具有相同键的值

In [12]:
pairRDD = sc.parallelize([("panda",0),("pink",3),("pirate",3),("panda",3),("pink",4)])
print pairRDD.collect()
print type(pairRDD)

[('panda', 0), ('pink', 3), ('pirate', 3), ('panda', 3), ('pink', 4)]
<class 'pyspark.rdd.RDD'>


In [27]:
#计算每个键对应的平均值
pairRDDmapValues = pairRDD.mapValues(lambda x:(x,1))#mapValues()函数，对pairRDD中每个值应用一个函数而不改变键
print pairRDDmapValues.collect()
pairRDDReduceByKey = pairRDDmapValues.reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1]))#reduceByKey()函数，合并具有相同键的值
print pairRDDReduceByKey.collect()

from __future__ import division
pairRDDAverageByKey = pairRDDReduceByKey.map(lambda x: x[1][0]/x[1][1])
print "每个键对应的平均值",pairRDDAverageByKey.collect()

[('panda', (0, 1)), ('pink', (3, 1)), ('pirate', (3, 1)), ('panda', (3, 1)), ('pink', (4, 1))]
[('pink', (7, 2)), ('panda', (3, 2)), ('pirate', (3, 1))]
pairRDDReduceByKey2: [('pink', (3, 1, 4, 1)), ('panda', (0, 1, 3, 1)), ('pirate', (3, 1))]
每个键对应的平均值 [3.5, 1.5, 3.0]


In [26]:
#(sum,count)
# 参考：https://github.com/mahmoudparsian/pyspark-tutorial/blob/master/tutorial/combine-by-key/spark-combineByKey.md
sumCount = pairRDD.combineByKey(lambda value: (value, 1),#createCombiner 参数对应PairRDD的value，
                            lambda x, value: (x[0] + value, x[1] + 1),#mergeValue参数一个combiner （sum,count）和 一个new value
                            lambda x, y: (x[0] + y[0], x[1] + y[1])#mergeCombiner,(sum, count)
                           )

averageByKey = sumCount.map(lambda (key, (totalSum, count)): (key, totalSum / count))
print averageByKey.collect()
print averageByKey.collectAsMap()

[('pink', 3.5), ('panda', 1.5), ('pirate', 3.0)]
{'pink': 3.5, 'panda': 1.5, 'pirate': 3.0}


#(sum,count)
#参考：https://github.com/mahmoudparsian/pyspark-tutorial/blob/master/tutorial/combine-by-key/spark-combineByKey.md
sumCount = pairRDD.combineByKey(lambda value: (value, 1),#createCombiner 参数对应PairRDD的value，
                            lambda x, value: (x[0] + value, x[1] + 1),#mergeValue参数一个combiner （sum,count）和 一个new value
                            lambda x, y: (x[0] + y[0], x[1] + y[1])#mergeCombiner,(sum, count)
                           )

averageByKey = sumCount.map(lambda (key, (totalSum, count)): (key, totalSum / count))
print averageByKey.collect()
print averageByKey.collectAsMap()

In [23]:
#单词计数
lines.collect()
words = lines.flatMap(lambda x:x.split())
words.collect()
wordsMap = words.map(lambda x:(x,1))#单词为键，1构成的pair RDD
wordsMap.collect()
wordsCount = wordsMap.reduceByKey(lambda x,y:x+y)
wordsCount.collect()

#countByvalue()单词计数
countbyValue = words.countByValue()
type(countbyValue)
print countbyValue

defaultdict(<type 'int'>, {u'all': 5, u'looked': 4, u'cried': 4, u'better,': 1, u'top': 1, u'"Why': 1, u'liked': 1, u'fair!"': 1, u'comfortable,': 1, u'down.': 1, u'view,': 1, u'lives.': 1, u'Until': 1, u'ground': 1, u'up.': 1, u'time,': 1, u'His': 1, u'cliff,': 1, u'just': 1, u'mother': 10, u'had': 4, u'him,': 1, u'him': 1, u'to': 11, u'only': 1, u'going': 2, u'mother.': 1, u'enjoying': 1, u'morsels3': 1, u'anger.': 1, u'faster.': 2, u'do': 3, u'good': 1, u'"Come': 1, u'Faster': 1, u'get': 2, u'food': 3, u'Many': 1, u'trees': 1, u'It': 3, u'craved4.': 1, u'She': 2, u'mother,"': 1, u'world': 1, u"I'm": 2, u'hungry.': 1, u'day': 2, u'Once': 1, u'and': 23, u'How': 1, u'like': 1, u'did': 1, u'die': 1, u'Over': 1, u'nourishing': 1, u'meal,"': 1, u'soon,': 2, u'stopped': 1, u'she': 6, u'always': 1, u'where': 1, u'vision': 1, u'few': 1, u'"How?"': 1, u'nest.': 3, u'"You\'re': 1, u'some': 1, u'idea': 1, u'up': 4, u'perched': 1, u'picked': 1, u'die."': 1, u'whined': 2, u'me?': 1, u'out': 1, u'

    并行度调优

In [30]:
#reduceByKey()
data = sc.parallelize([("a",3),("b",4),("a",1)])
print data.reduceByKey(lambda x,y:x+y).collect()
print data.reduceByKey(lambda x,y:x+y,10).collect()

[('a', 4), ('b', 4)]
[('b', 4), ('a', 4)]


    4.3.2数据分组
    groupByKey根据pairRDD的键对数据进行分组，RDD(K,V)->RDD(K,Iterable(V))
    cogroup()函数对多个共享同一个键的RDD进行分组，例如RDD(K,V),RDD(K,W)>RDD(K,(Iterable(V),Iterable(W)))

In [3]:
data = sc.parallelize([("a",3),("b",4),("a",1)])
data.groupByKey().collect()

[('a', <pyspark.resultiterable.ResultIterable at 0x6d3a6a0>),
 ('b', <pyspark.resultiterable.ResultIterable at 0x6d3a710>)]

    4.3.3连接    PairRDD之间用键连接起来，常用连接方式左外连接，右外连接，内连接，交叉连接

In [9]:
data1 = sc.parallelize([("a",3),("b",4),("c",1)])
data2 = sc.parallelize([("a",2),("b",5),("d",6)])
print data1.join(data2).collect()#内连接
print data1.leftOuterJoin(data2).collect()#左外连接
print data1.rightOuterJoin(data2).collect()#右外连接

[('a', (3, 2)), ('b', (4, 5))]
[('a', (3, 2)), ('c', (1, None)), ('b', (4, 5))]
[('a', (3, 2)), ('b', (4, 5)), ('d', (None, 6))]


    4.3.4数据排序
    RDD.sortByKey(ascending=True, numPartitions=None, keyfunc=<function <lambda> at 0x7fc35dbcf848)

In [14]:
tmp = [('a', 1), ('b', 2), ('1', 3), ('d', 4), ('2', 5)]
print sc.parallelize(tmp).sortByKey().collect()
tmp2 = [('Mary', 1), ('had', 2), ('a', 3), ('little', 4), ('lamb', 5)]
tmp2.extend([('whose', 6), ('fleece', 7), ('was', 8), ('white', 9)])
print sc.parallelize(tmp2).sortByKey(False, 3, keyfunc=lambda k: k.upper()).collect()

[('1', 3), ('2', 5), ('a', 1), ('b', 2), ('d', 4)]
[('whose', 6), ('white', 9), ('was', 8), ('Mary', 1), ('little', 4), ('lamb', 5), ('had', 2), ('fleece', 7), ('a', 3)]


    4.4PairRDD行动操作
    所有基础RDD的行动操作都可应用于PairRDD，另外PairRDD也有专属的行动操作
    countByKey()每个键对应的元素计数，返回字典
    collectAsMap()将结果以字典的形式返回，以便查询

In [21]:
result1 = sc.parallelize([(1,2),(3,4),(3,6)]).countByKey()#defaultdict(int, {1: 1, 3: 2})
result2 = sc.parallelize([(1,2),(3,4),(3,6)]).collectAsMap()
print result1
print result2
print result2[1]
print result2[3]

defaultdict(<type 'int'>, {1: 1, 3: 2})
{1: 2, 3: 6}
2
6


    4.5数据分区